# 🤖 Análise de Consenso entre LLMs
## Notebook Refatorado com Alternative Params

Este notebook usa:
- Componentes modulares
- Logging com loguru
- Integração com HuggingFace
- **Alternative params** para testar variações

## 1️⃣ Setup e Configuração

In [1]:
# Imports
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger

# Configurar logging
logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

logger.success("✓ Setup completo")

14:02:06 | SUCCESS  | ✓ Setup completo


## 2️⃣ Carregar Dataset do HuggingFace

In [2]:
from src.utils.data_loader import load_hf_dataset, load_hf_dataset_as_dataframe, list_available_datasets

# Listar datasets
logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

14:02:06 | Datasets disponíveis:
14:02:06 |   - agnews
14:02:06 |   - mpqa
14:02:06 |   - webkb
14:02:06 |   - ohsumed
14:02:06 |   - acm
14:02:06 |   - yelp_2013
14:02:06 |   - dblp
14:02:06 |   - books
14:02:06 |   - reut90
14:02:06 |   - wos11967
14:02:06 |   - twitter
14:02:06 |   - trec
14:02:06 |   - wos5736
14:02:06 |   - sst1
14:02:06 |   - pang_movie
14:02:06 |   - movie_review
14:02:06 |   - vader_movie
14:02:06 |   - subj
14:02:06 |   - sst2
14:02:06 |   - yelp_reviews
14:02:06 |   - 20ng
14:02:06 |   - medline


In [3]:
# Carregar dataset
dataset_name = "agnews"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(dataset_name)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

14:02:06 | Carregando dataset: agnews
14:02:12 | Split 'train': 510400 exemplos
14:02:12 | Categorias extraídas automaticamente: [0, 1, 2, 3]
14:02:12 | Amostra reduzida para 100 exemplos
14:02:12 | Coluna de texto: text
14:02:12 | Ground truth carregado da coluna 'label'
14:02:12 | Textos: 100
14:02:12 | Categorias: [0, 1, 2, 3]
14:02:12 | Ground truth: Sim


In [4]:
# Visualizar amostra
logger.info("Amostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

14:02:12 | Amostra dos textos:
14:02:12 | 1. "Irish Claim 2nd Title","Notre Dame goalkeeper Erika Bohn seals the Irish's second NCAA championship...
14:02:12 |    Label: 1
14:02:12 | 2. "Court rules for BC in flap over exit fee","Boston College cleared a major legal hurdle in its bid t...
14:02:12 |    Label: 1
14:02:12 | 3. "Scientific Method Man","Gordon Rugg cracked the 400-year-old mystery of the Voynich manuscript. Nex...
14:02:12 |    Label: 3


In [5]:
df, categories = load_hf_dataset_as_dataframe(dataset_name)

14:02:12 | Carregando dataset: agnews
14:02:17 | Split 'train': 510400 exemplos
14:02:17 | Categorias extraídas automaticamente: [0, 1, 2, 3]
14:02:17 | Amostra reduzida para 100 exemplos
14:02:17 | Coluna de texto: text
14:02:17 | Ground truth carregado da coluna 'label'
14:02:17 | DataFrame criado com 100 linhas


In [6]:
categories

[0, 1, 2, 3]

## 3️⃣ Configurar Modelos LLM

### Opção A: Usar apenas parâmetros padrão (temp=0)

In [7]:
from src.llm_annotation_system.annotation.llm_annotator import LLMAnnotator
from src.experiments.base_experiment import DEFAULT_MODELS, BASE_ANNOTATION_PROMPT

# Inicializar SEM alternative params
annotator = LLMAnnotator(
    dataset_name=dataset_name,
    prompt_template=BASE_ANNOTATION_PROMPT,
    models=DEFAULT_MODELS,
    categories=categories,
    api_keys=None,
    use_langchain_cache=True,
    use_alternative_params=False  # Apenas temp=0
)

logger.success(f"✓ Annotator inicializado com {len(annotator.models)} modelos")

14:02:18 | Cache carregado: 30 entradas
14:02:18 | Cache LangChain ativado: ..\..\data\.cache\langchain\langchain_cache.db
14:02:18 | Template do prompt preparado
14:02:18 | LLMAnnotator inicializado
14:02:18 | Modelos: 5 | Categorias: 4
14:02:18 | ✓ Annotator inicializado com 5 modelos


In [8]:
annotator.annotation_engine.template

'You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: world\n- 1: sports\n- 2: business\n- 3: Sci/Tech\n\n**Text to classify:**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category number as your response\n- Do not include explanations\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across similar texts\n\n**Your classification (category number only):**'

### Opção B: Usar alternative params (temp=0, 0.3, 0.5)

**Atenção**: Isso cria 9 modelos (3 base + 6 variações)

In [9]:
# Descomente para usar alternative params:

# annotator = LLMAnnotator(
#     models=models,
#     categories=categories,
#     api_keys=None,
#     use_langchain_cache=True,
#     use_alternative_params=True  # Expande para 9 modelos
# )

# logger.success(f"✓ Annotator com alternative params: {len(annotator.models)} modelos")
# logger.info(f"  Modelos expandidos: {annotator.models}")

## 4️⃣ Executar Anotação

### Testar anotação

In [10]:
df

,text,label,label_description
0,"""Irish Claim 2nd Title"",""Notre Dame goalkeeper...",1,sports
1,"""Court rules for BC in flap over exit fee"",""Bo...",1,sports
2,"""Scientific Method Man"",""Gordon Rugg cracked t...",3,Sci/Tech
3,"""Money From Salvadoran Immigrants Aids Farming...",2,business
4,"""U.S. Pounds Falluja Diehards, Violence in Nor...",0,world
...,...,...,...
95,"""Fannie Mae in Deal to Up Capital"","" WASHINGTO...",2,business
96,"""IBM, AMD develop new use of strained silicon""...",3,Sci/Tech
97,"""Schumacher takes second best in opening sessi...",1,sports
98,"""The cream of the crop"",""Their beacons of hope...",1,sports


In [ ]:
# Parâmetros
num_repetitions = 3
text = texts[0]
model = "qwen3-8b"

# Estimativa
total_annotation = len(text) * num_repetitions
logger.warning(f"  Modelo: {model}")
logger.warning(f"  Texto: {len(text)}")
logger.warning(f"  Repetições: {num_repetitions}")

# Anotar
annotations = annotator.annotate_single(
    text=text,
    model=model,
    num_repetitions=num_repetitions,
    use_cache=False
)

logger.success("✓ Anotação completa")

annotations

14:02:18 |   Modelo: deepseek-r1-8b
14:02:18 |   Texto: 191
14:02:18 |   Repetições: 3


content='\n1' response_metadata={'model': 'deepseek-r1:8b', 'created_at': '2025-11-30T17:02:27.6361282Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 7177027300, 'load_duration': 2300431900, 'prompt_eval_count': 482, 'prompt_eval_duration': 45010800, 'eval_count': 3, 'eval_duration': 40748800} id='run-6ad79fea-6e3f-4cfc-a643-dc73192e7281-0'


14:02:28 | ✓ Anotação completa


content='\n1' response_metadata={'model': 'deepseek-r1:8b', 'created_at': '2025-11-30T17:02:27.6361282Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 7177027300, 'load_duration': 2300431900, 'prompt_eval_count': 482, 'prompt_eval_duration': 45010800, 'eval_count': 3, 'eval_duration': 40748800} id='run-6ad79fea-6e3f-4cfc-a643-dc73192e7281-0'
content='\n1' response_metadata={'model': 'deepseek-r1:8b', 'created_at': '2025-11-30T17:02:27.6361282Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 7177027300, 'load_duration': 2300431900, 'prompt_eval_count': 482, 'prompt_eval_duration': 45010800, 'eval_count': 3, 'eval_duration': 40748800} id='run-6ad79fea-6e3f-4cfc-a643-dc73192e7281-0'


[1, 1, 1]

### Anotando dataset completo

In [12]:
# Parâmetros
num_repetitions = 1

# Anotar
df_annotations = annotator.annotate_dataset(
    texts=texts,
    num_repetitions=num_repetitions,
    use_cache=False
)

logger.success("✓ Anotações completas")
display(df_annotations.head())

14:02:28 | Iniciando anotação
14:02:28 | Textos: 100 | Modelos: 5 | Repetições: 1
14:02:28 | Total de anotações: 500
Anotando:   0%|          | 0/100 [00:00<?, ?it/s]14:02:28 | Resposta é None ou ''


content='\n1' response_metadata={'model': 'deepseek-r1:8b', 'created_at': '2025-11-30T17:02:27.6361282Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 7177027300, 'load_duration': 2300431900, 'prompt_eval_count': 482, 'prompt_eval_duration': 45010800, 'eval_count': 3, 'eval_duration': 40748800} id='run-6ad79fea-6e3f-4cfc-a643-dc73192e7281-0'
content='' response_metadata={'model': 'qwen3:8b', 'created_at': '2025-11-30T16:18:26.7282287Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'length', 'total_duration': 4311233300, 'load_duration': 2293494600, 'prompt_eval_count': 238, 'prompt_eval_duration': 133626800, 'eval_count': 100, 'eval_duration': 1825898800} id='run-9dab1461-25b0-44fa-8218-e5610d49f165-0'
content='1\n' response_metadata={'model': 'gemma3:4b', 'created_at': '2025-11-30T15:45:13.5630884Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_du

Anotando:   1%|          | 1/100 [00:04<08:10,  4.96s/it]


KeyboardInterrupt: 

## 5️⃣ Calcular Consenso

In [ ]:
# Calcular consenso
df_with_consensus = annotator.calculate_consensus(df_annotations)

# Estatísticas
logger.info("\n📊 Estatísticas de Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Mediana: {df_with_consensus['consensus_score'].median():.2%}")
logger.info(f"  Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

# Distribuição por nível
levels = df_with_consensus['consensus_level'].value_counts()
logger.info("\nDistribuição por nível:")
for level, count in levels.items():
    logger.info(f"  {level}: {count} ({count/len(df_with_consensus):.1%})")

In [ ]:
# Visualizações
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(df_with_consensus['consensus_score'], bins=20, edgecolor='black')
axes[0].set_xlabel('Consensus Score')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribuição de Scores de Consenso')

# Barras por nível
levels.plot(kind='bar', ax=axes[1], color=['green', 'orange', 'red'])
axes[1].set_xlabel('Nível de Consenso')
axes[1].set_ylabel('Contagem')
axes[1].set_title('Casos por Nível de Consenso')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 6️⃣ Análise de Alternative Params

**Nota**: Esta seção só funciona se `use_alternative_params=True`

In [ ]:
# Verificar se alternative params foi usado
if annotator.use_alternative_params:
    logger.info("📊 Analisando impacto dos alternative params...")
    
    # Agrupar por modelo base
    for base_model in models:
        # Encontrar variações deste modelo
        variations = [m for m in annotator.models if m.startswith(base_model)]
        
        logger.info(f"\n{base_model}:")
        
        for var in variations:
            if f'{var}_consensus_score' in df_with_consensus.columns:
                score = df_with_consensus[f'{var}_consensus_score'].mean()
                logger.info(f"  {var}: {score:.2%} consenso interno")
    
    # Comparar temperaturas
    fig, ax = plt.subplots(figsize=(12, 6))
    
    consensus_cols = [col for col in df_with_consensus.columns 
                     if '_consensus_score' in col and '_alt' in col or 
                     (col.replace('_consensus_score', '') in models)]
    
    if consensus_cols:
        means = [df_with_consensus[col].mean() for col in consensus_cols]
        labels = [col.replace('_consensus_score', '') for col in consensus_cols]
        
        ax.bar(labels, means)
        ax.set_ylabel('Consenso Interno Médio')
        ax.set_title('Consenso por Variação de Parâmetros')
        ax.tick_params(axis='x', rotation=45)
        plt.tight_layout()
        plt.show()
    
else:
    logger.warning("Alternative params não foi usado. Para análise detalhada, reinicialize com use_alternative_params=True")

## 7️⃣ Análise Detalhada de Consenso

In [ ]:
from src.llm_annotation_system.consensus.consensus_analyzer import ConsensusAnalyzer

# Inicializar analyzer
analyzer = ConsensusAnalyzer(categories)

# Colunas de consenso
consensus_cols = [col for col in df_with_consensus.columns if '_consensus' in col and '_score' not in col]

logger.info(f"Analisando {len(consensus_cols)} anotadores")

# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus,
    annotator_cols=consensus_cols,
    output_dir="./results"
)

logger.success("✓ Relatório gerado")

In [ ]:
# Métricas
logger.info("\n📊 Métricas de Concordância:")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

# Interpretação
kappa = report['fleiss_kappa']
if kappa > 0.8:
    logger.success("Concordância excelente!")
elif kappa > 0.6:
    logger.info("Concordância boa")
elif kappa > 0.4:
    logger.warning("Concordância moderada")
else:
    logger.warning("Concordância fraca")

In [ ]:
# Matriz de concordância
agreement_df = report['pairwise_agreement']

plt.figure(figsize=(12, 10))
sns.heatmap(agreement_df, annot=True, fmt='.2f', cmap='YlGnBu', cbar_kws={'label': 'Agreement'})
plt.title('Matriz de Concordância Par a Par')
plt.tight_layout()
plt.show()

In [ ]:
# Casos problemáticos
problematic = report.get('problematic_cases')
if problematic is not None and len(problematic) > 0:
    logger.warning(f"\n⚠️  {len(problematic)} casos problemáticos identificados")
    display(problematic.head())
else:
    logger.success("\n✓ Nenhum caso problemático identificado")

## 8️⃣ Validação com Ground Truth

In [ ]:
if ground_truth:
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    # Adicionar ground truth
    df_with_consensus['ground_truth'] = ground_truth
    
    # Accuracy
    accuracy = accuracy_score(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    logger.success(f"\n🎯 Accuracy: {accuracy:.2%}")
    
    # Classification report
    logger.info("\nClassification Report:")
    print(classification_report(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    ))
    
    # Confusion matrix
    cm = confusion_matrix(
        df_with_consensus['ground_truth'],
        df_with_consensus['most_common_annotation']
    )
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix vs Ground Truth')
    plt.tight_layout()
    plt.savefig('./results/confusion_vs_ground_truth.png', dpi=150)
    plt.show()
    
else:
    logger.info("\n⚠️  Ground truth não disponível - pulando validação")

## 9️⃣ Exportar Resultados

In [ ]:
import json
from pathlib import Path

# Criar diretório
results_dir = Path('./results/final')
results_dir.mkdir(parents=True, exist_ok=True)

# Salvar CSVs
df_with_consensus.to_csv(results_dir / 'dataset_anotado_completo.csv', index=False)
logger.info(f"✓ Salvos: {len(df_with_consensus)} registros")

# Alta confiança
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf.to_csv(results_dir / 'alta_confianca.csv', index=False)
logger.info(f"✓ Alta confiança: {len(high_conf)} registros")

# Necessita revisão
low_conf = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
low_conf.to_csv(results_dir / 'necessita_revisao.csv', index=False)
logger.info(f"✓ Necessita revisão: {len(low_conf)} registros")

# Sumário JSON
summary = {
    'dataset': {
        'name': dataset_name,
        'total_texts': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None
    },
    'config': {
        'models': models,
        'total_models': len(annotator.models),
        'use_alternative_params': annotator.use_alternative_params,
        'num_repetitions': num_repetitions,
        'total_annotations': len(texts) * len(annotator.models) * num_repetitions
    },
    'results': {
        'consensus_mean': float(df_with_consensus['consensus_score'].mean()),
        'consensus_median': float(df_with_consensus['consensus_score'].median()),
        'high_consensus': int((df_with_consensus['consensus_level'] == 'high').sum()),
        'medium_consensus': int((df_with_consensus['consensus_level'] == 'medium').sum()),
        'low_consensus': int((df_with_consensus['consensus_level'] == 'low').sum()),
    },
    'metrics': {
        'fleiss_kappa': float(report['fleiss_kappa']),
        'fleiss_interpretation': report['fleiss_interpretation']
    }
}

if ground_truth:
    summary['validation'] = {
        'accuracy': float(accuracy)
    }

with open(results_dir / 'sumario_experimento.json', 'w') as f:
    json.dump(summary, f, indent=2)

logger.success("\n✓ Resultados exportados com sucesso!")

## 🔟 Resumo Final

In [ ]:
logger.info("\n" + "="*80)
logger.success("RESUMO DO EXPERIMENTO")
logger.info("="*80)

logger.info(f"\n📊 Dataset: {dataset_name}")
logger.info(f"  Textos: {len(texts)}")
logger.info(f"  Categorias: {len(categories)}")

logger.info(f"\n🤖 Configuração:")
logger.info(f"  Modelos base: {len(models)}")
logger.info(f"  Total modelos: {len(annotator.models)}")
logger.info(f"  Alternative params: {annotator.use_alternative_params}")
logger.info(f"  Repetições: {num_repetitions}")

logger.info(f"\n📈 Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

if ground_truth:
    logger.info(f"\n🎯 Validação:")
    logger.info(f"  Accuracy: {accuracy:.2%}")

logger.info(f"\n📁 Arquivos gerados em: {results_dir}/")

cache_stats = annotator.get_cache_stats()
logger.info(f"\n💾 Cache: {cache_stats['total_entries']} entradas")

logger.success("\n✅ Análise completa!")